company-name 公司名稱
job-opening 職缺
requirement 應備技能

In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import json
import pandas as pd

#os.mkdir("./104_job")

headers = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36",
    "Referer": "https://www.104.com.tw/job"
}


url = "https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=%E8%B3%87%E6%96%99%E5%88%86%E6%9E%90%E5%B8%AB&jobcatExpansionType=0&order=14&asc=0&page={}&mode=s&jobsource=2018indexpoc"
page = 1

#scraw page 1~3
for i in range(0,3):                                       
    res = requests.get(url.format(page),headers=headers)

    html = res.text

    soup = BeautifulSoup(html, "html.parser")

    job = soup.select("article[class='b-block--top-bord job-list-item b-clearfix js-job-item']")
    job_link = soup.select("a[class='js-job-link']")
    
    #scraw job_url company_name skill description
    for j , k in zip(job, job_link):
        company_name = j["data-cust-name"]
        job_name = j["data-job-name"]
        job_url = "https:" + k["href"]
        
        job_js_url = "https://www.104.com.tw/job/ajax/content/{}"

        job_res = requests.get(url=job_js_url.format(k["href"][21:26]), headers=headers)

        job = json.loads(job_res.text)
        
        area = job["data"]["jobDetail"]["addressRegion"]
        
        job_Description = job["data"]["jobDetail"]["jobDescription"]
        job_Description = job_Description.replace(",","，")

        job_skill = ""
        tmp_skill = job["data"]["condition"]["specialty"]

        for i in range(len(tmp_skill)):
            job_skill = job_skill + " " + tmp_skill[i]["description"]
        
        others = job["data"]['condition']['other']
        
        #save as txt_file
        try:
            with open("./104_job/{}.txt".format(company_name),"w",encoding="utf-8") as f:
                f.write(company_name+"\n="+job_name+"\n="+area+"\n="+job_Description+"\n="+job_skill+"\n="+others+"\n="+job_url)
                
        except OSError:
            pass
        print(company_name)
        print(area)
        print(job_name)
        print(job_url)
        print(job_Description)
        print(job_skill)
        print(others)
        print("=="*20)

    page += 1

    
file_list = os.listdir("./104_job")
df = pd.DataFrame(columns = ["company_name","job_name","area","job_Description","skills","others","104_url"])

#open txt_file append and save as csv_file
job_row_data = []
for each_job in file_list:
    with open("./104_job"+"/"+each_job, "r", encoding="utf-8") as f:
        tmp_job_content = f.read()
    job_content = tmp_job_content.replace("\n","")
    tmp_job_row_data = job_content.split("=")
    job_row_data.append(tmp_job_row_data)
#print(job_row_data)

df = df.append(pd.DataFrame(job_row_data,columns = ["company_name","job_name","area","job_Description","skills","others","104_url"]))
df.to_csv("./job.csv",index=0, encoding="utf-8")
